# Vulnerability Analysis and Defense Generation using AIShield SDK
* File Name: Reference_implementation of BFSI 
* Date of creation(dd-mm-yyyy) : 27-01-2023
* Author Name/Dept : AIShield
* Organization : BGSW
* Description : Source Code of reference implementation of AIShield API
* Copyright : Copyright 2022 Bosch Global Software Technologies Private Limited. All Rights Reserved.

### Input
This Example does the following:
* Trains a XGBOOST Model for Tabular Classification on the Banking Marketing Campagin
* Trained Model File : "_xgboost_tc_model.pkl_"

### Output and Artifacts
1. Vulnerability report
2. Defense report
3. Defense model
4. attack samples


In [ ]:
# # Install AIShield SDK.
# # If not already installed, install by uncommenting the code below

# !pip install aishield

In [ ]:
"""
Description: commands to install all the packages, remove comments to install all the libraries
Known issue: In colab, if get matplotlib error: cannot import name '_png' from 'matplotlib'. Use !pip install matplotlib==3.1.3
"""
# ! pip install xgboost==1.6.2
# ! pip install pandas==1.5.1
# ! pip install scikit-learn==1.1.3
# ! pip install numpy==1.23.4
# ! pip install pyminizip
# !pip install requests==2.28.0
# !pip install humanfriendly==9.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 17.8 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2


# 1.0 Importing libraries

In [ ]:
"""
Description: Import libraries
"""

import os
import copy
import json
import time
import pickle
import shutil
import requests
import csv
import py_compile
import pyminizip
import zipfile
import numpy as np
import pandas as pd
from sklearn import metrics
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

#importing AIShield Library
import aishield as ais

# 2.0 Data Loading and Preprocessing

##### Download bank-additional.zip dataset from the link https://archive.ics.uci.edu/ml/machine-learning-databases/00222/ and extract it. For this tutorial, file is already downloaded. If running in colab, copy the folder to colab under 'files'

In [ ]:
'''
Description : Loading credit card fraud detection dataset
'''
df = pd.read_csv('bank-additional/bank-additional-full.csv',quoting=csv.QUOTE_ALL, delimiter=";")

In [ ]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [ ]:
df.shape

(41188, 21)

In [ ]:
df.replace(['basic.6y','basic.4y', 'basic.9y'], 'basic', inplace=True)

In [ ]:
columns = ['day_of_week','month']
df.drop(columns, axis=1, inplace=True)

In [ ]:
'''Description : Label Encoding the Categorical Columns'''

le =LabelEncoder()
df.job = le.fit_transform(df.job)
df.marital = le.fit_transform(df.marital)
df.education = le.fit_transform(df.education)
df.housing = le.fit_transform(df.housing)
df.loan = le.fit_transform(df.loan)
df.poutcome = le.fit_transform(df.poutcome)
df.contact = le.fit_transform(df.contact)
df.default = le.fit_transform(df.default)
df.y = le.fit_transform(df.y)

In [ ]:
df.head()

,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,3,1,0,0,0,0,1,261,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
1,57,7,1,1,1,0,0,1,149,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
2,37,7,1,1,0,2,0,1,226,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
3,40,0,1,0,0,0,0,1,151,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0
4,56,7,1,1,0,0,2,1,307,1,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0


In [ ]:
output = 'y'
 
X = df.loc[:, df.columns != output]
y = df['y']

In [ ]:
'''
Description : Splitting data for validation
'''
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)

In [ ]:
'''
Description : Check size of dataset
'''
print("shape of x_train: ",X_train.shape)
print("shape of y_train: {}".format(y_train.shape))
print(f'shape of x_test: {X_test.shape}')
print(f'shape of y_test: {y_test.shape}')

shape of x_train:  (30891, 18)
shape of y_train: (30891,)
shape of x_test: (10297, 18)
shape of y_test: (10297,)


# 3.0 Model Development and Training

In [ ]:
def make_directory(directory):
    """
    create directory

    Parameters
    ----------
    directorys : list containing the directorys path to create 
    Returns
    -------
    None.

    """
    for d in directory:
        if os.path.isdir(d):
            print("directory {} already exist".format(d))
        if os.path.isdir(d)==False:
            os.mkdir(path=d)
            print("directory {} created successfully".format(d))

In [ ]:
def delete_directory(directorys):
    """
    delete directory 

    Parameters
    ----------
    directorys : list containing the directorys to deleate along with all the files

    Returns
    -------
    None.

    """
    if len(directorys)>=1:
        for d in directorys:
            if os.path.isdir(d):
                try:
                    if os.path.isfile(d):
                        os.remove(path=d)
                    else:
                        shutil.rmtree(path=d)
                        print("Removed: {}".format(d))
                except:
                    print("Failed to removed: {}".format(d))
                

In [ ]:
def make_archive(base_name,root_dir,zip_format='zip'):
    """
    created zip for given folder

    Parameters
    ----------
    base_name : name of zip file
    root_dir : directory to archive/zip
    zip_format : zip or tar 
        DESCRIPTION. The default is 'zip'.

    Returns
    -------
    None.

    """
    shutil.make_archive(base_name=base_name, format=zip_format, root_dir=root_dir)
    

In [ ]:
"Description : Create data, model and label folder"
data_path=os.path.join(os.getcwd(),"data")
model_path=os.path.join(os.getcwd(),"model")
minmax_path=os.path.join(os.getcwd(),"minmax")
zip_path=os.path.join(os.getcwd(),"zip")
pyc_model_path=os.path.join(os.getcwd(),"pycmodel")
report_path = os.path.join(os.getcwd(), "reports")
#deleting folder
delete_directory(directorys=[data_path,model_path,minmax_path,zip_path,pyc_model_path,report_path])

#creating folder
make_directory([data_path,model_path,minmax_path,zip_path,pyc_model_path,report_path])

directory /content/data created successfully
directory /content/model created successfully
directory /content/minmax created successfully
directory /content/zip created successfully
directory /content/pycmodel created successfully
directory /content/reports created successfully


In [ ]:
def model():
    XG = XGBClassifier()
    return XG

In [ ]:
"""
Description : Create architecture of the model
"""
model=model()
print(model)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None,
              reg_alpha=None, reg_lambda=None, ...)


In [ ]:
"""
Description: Trainig the model 
"""
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [ ]:
"""
Saving the model under model directory
"""
modelname = "xgboost_tc_model.pkl"
pickle.dump(model, open(model_path+"/"+modelname, 'wb'))

In [ ]:
"""
Description: Get prediction on validation data
"""
pred=model.predict(X_test)

In [ ]:
print('Accuracy score of the Decision Tree model is {}'.format(accuracy_score(y_test, pred)))
print('F1 score of the Decision Tree model is {}'.format(f1_score(y_test, pred)))
print('Confusion Matrix \n {}'.format(confusion_matrix(y_test, pred)))
print('Classification Report \n {}'.format(metrics.classification_report(y_test, pred)))

Accuracy score of the Decision Tree model is 0.9159949499854326
F1 score of the Decision Tree model is 0.601932811780948
Confusion Matrix 
 [[8778  354]
 [ 511  654]]
Classification Report 
               precision    recall  f1-score   support

           0       0.94      0.96      0.95      9132
           1       0.65      0.56      0.60      1165

    accuracy                           0.92     10297
   macro avg       0.80      0.76      0.78     10297
weighted avg       0.91      0.92      0.91     10297



# 4.0 Prepare Data , Model and MinMax

In [ ]:
"""
Description: Zip data
"""
df.to_csv(os.path.join(data_path,"banking_final_dataset.csv"),index=False)
make_archive(base_name=os.path.join(zip_path,"data"),root_dir=data_path,zip_format='zip')

In [ ]:
"""Prepare MinMax"""

min_values = df.min().to_numpy()
max_values = df.max().to_numpy()
x= np.array([min_values,max_values])
df_m = pd.DataFrame(x,columns=df.columns)
df_m.to_csv(minmax_path+"/minmax.csv",index=False)

In [ ]:
"""
Description: Zip Minmax
"""
make_archive(base_name=os.path.join(zip_path,"minmax"),root_dir=minmax_path,zip_format='zip')

In [ ]:
"""
Description: Create encrypted Model
"""
def create_model_pyc(model, model_framework,pyc_model_path):
    
    if model_framework.lower() == "scikit-learn":

        python_code ='''
#importing libraries
import pickle
import zipfile
import os

"""
    class for base model
"""
#define class
class BaseModel():
    def __init__(self,model_path=""):

        """
        constructor for class

        Parameters
        ----------
        input_shape : TYPE, optional
            DESCRIPTION. The default is (100,1).
        Returns
        -------
        None.

        """
        self.model_path = model_path
        
    def load_protected_pickleModel(self,filename,password,picklemodelname):
      

        """
        model architecture

        Parameters
        ----------
        filename : string
            DESCRIPTION.zipped protected filename
            
        password : String
            DESCRIPTION.zipped file password
        
        picklemodelname  :String
            DESCRIPTION.pickle file name present in a ziiped protected
            
        Returns
        -------
        model : model
            DESCRIPTION.

        """
        filepath = os.path.join(self.model_path,filename)
        #print(filepath)
        #print(filename)
        with zipfile.ZipFile(filepath, 'r') as file:
            with file.open(picklemodelname,'r',pwd = bytes(password, 'utf-8')) as f:
                pck = pickle.load(f)
        return pck
    
    def predict(self,X):

        """
        predict for given data

        Parameters
        ----------
        X : numpy array 
            DESCRIPTION.

        Returns
        -------
        pred : numpy array
            DESCRIPTION.

        """
        filename = "modelencrypt.zip"
        password = "987654321"
        picklemodelname = "xgboost_tc_model.pkl"
        model = self.load_protected_pickleModel(filename=filename,password=password,picklemodelname=picklemodelname)
        pred = model.predict(X)
        return pred'''
        # Writing to file
        with open("base_model.py", "w") as file:
            # Writing data to a file
            file.writelines(python_code)
         
        """
        Description: function to create .pyc file
        """
        py_compile.compile(file="base_model.py",cfile=pyc_model_path+'/base_model.pyc')
        
        """
        Description: function to create zipped password protected pickle model file
        """

        def zip_model(input_path,output_path,password,com_lvl=5):
            pyminizip.compress(input_path, None, output_path,
                               password, com_lvl)
            
        zip_model(input_path =os.path.join(model_path,modelname),output_path = pyc_model_path+"/modelencrypt.zip" ,password = "987654321",com_lvl=5)
            

In [ ]:
"""
Description: Creating encrypted Model
"""
model_framework = "scikit-learn"
model_encryption = 0 #0 for non encrypted file and 1 for encrypted (pyc) 
create_model_pyc(model,model_framework, pyc_model_path)

In [ ]:
"""
Description: Zip model
"""
 #0 if model is uploaded directly as a zip, 1 if model is encryted as .pyc and uploaded as a zip
if os.path.isfile(os.path.join(zip_path,"model.zip")):
    delete_directory(directorys=[os.path.join(zip_path,"model.zip")])
if model_encryption:
    make_archive(base_name=os.path.join(zip_path,"bfsi_pyc_model"),root_dir=pyc_model_path,zip_format='zip')
else:
    make_archive(base_name=os.path.join(zip_path,"model"),root_dir=model_path,zip_format='zip')

# 5.0 Vulnerability Analysis and Defense Generation using AIShield SDK

In [ ]:
"""
Description: AIShield API URL and subscription key
""" 
url="https://xxxxxx/AIShieldAPI"
api_key = "xxxx"
org_id = "xxxxxxxxxxxx"

In [ ]:
"""
Description: File paths
"""
data_path=os.path.join(zip_path,'data.zip') #full path of data zip
label_path=os.path.join(zip_path,'minmax.zip') #full path of label(minmax) zip
model_path=os.path.join(zip_path,'model.zip') #full path of model zip

### `Perform Vulnerability Analysis through VulConfig Function`

In [ ]:
"""
Description: Initialize the AIShield API
"""
client = ais.AIShieldApi(api_url=url, api_key=api_key, org_id=org_id)

In [ ]:
"""
Description: Define the task and analysis type
"""
task_type = ais.get_type("task", "tabular_classification")
analysis_type = ais.get_type("analysis", "extraction")

In [ ]:
"""
Description: Perform model registration and upload the input artifacts
"""
status, job_details = client.register_model(task_type=task_type, analysis_type=analysis_type)
model_id = job_details.model_id
# print('Model id: {} \nInput artifacts will be uploaded as:\n data_upload_uri: {}\n minmax_upload_uri: {}'
#       '\n model_upload_uri: {}'.format(model_id, job_details.data_upload_uri, job_details.minmax_upload_uri,
#                                        job_details.model_upload_uri))

upload_status = client.upload_input_artifacts(job_details=job_details,
                                              data_path=data_path,
                                              minmax_path=label_path,
                                              model_path=model_path, )
print('Upload status: {}'.format(', '.join(upload_status)))

Upload status: data file upload successful, minmax file upload successful, model file upload successful


In [ ]:
"""
Description: Specify the appropriate configs required for vulnerability analysis
"""

vuln_config = ais.VulnConfig(task_type=task_type,
                             analysis_type=analysis_type,
                             defense_generate=True)

vuln_config.input_dimensions = (41188, 18)  # input dimension of banking marketing dataset
vuln_config.number_of_classes = 2  # number of classes of banking marketing dataset
vuln_config.is_category_columns = "yes"  # if cetgorical columns are present
vuln_config.categorical_columns_info = ["job","marital","education","default","housing","loan","contact","poutcome"]  # name of categorical columns
vuln_config.attack_type = "blackbox"  # greybox or blackbox depending upon the availability of information about Model, Data and Parameters
vuln_config.number_of_attack_queries = 60000  # Number of attack queries to be generated for testing model vulnerability 
vuln_config.encryption_strategy = 0  # value 0 (or) 1, if model is unencrypted or encrypted(pyc) respectively


print('TC-Extraction parameters are: \n {} '.format(vuln_config.get_all_params()))

TC-Extraction parameters are: 
 {'attack_type': 'blackbox', 'categorical_columns_info': 'job,marital,education,default,housing,loan,contact,poutcome', 'defense_bestonly': 'no', 'encryption_strategy': 0, 'input_dimensions': '(41188, 18)', 'is_category_columns': 'yes', 'model_api_details': '', 'model_framework': 'scikit-learn', 'normalize_data': 'yes', 'number_of_attack_queries': 60000, 'number_of_classes': 2, 'use_model_api': 'no', 'vulnerability_threshold': 0} 


In [ ]:
"""
Description: Run vulnerability analysis
"""
my_status, job_details = client.vuln_analysis(model_id=model_id, vuln_config=vuln_config)
my_job_id = job_details.job_id
print('status: {} \nJob_id: {} '.format(my_status, my_job_id))

status: success 
Job_id: gAAAAABj4NUdnijBn1ZMak9NbGuy-kpb-FxHHQji0CbYnr1CJvwY1ieLOrPbkWzQNhMfTj_3JlWr60OPYYriY-Tvejoz91g6Qg== 


In [ ]:
"""
Description: Monitor progress for given Job ID using the Link below
"""
# print('Click on the URL to view Vulnerability Dashboard (GUI): {}'.format(job_details.job_monitor_uri))

'\nDescription: Monitor progress for given Job ID using the Link below\n'

In [ ]:
"""
Description: Fetch Job status using Job ID
"""
my_status = client.job_status (job_id = my_job_id)
print('job status ', my_status)

2023-02-06 10:23:26,840 - INFO - Fetching job details for job id gAAAAABj4NUdnijBn1ZMak9NbGuy-kpb-FxHHQji0CbYnr1CJvwY1ieLOrPbkWzQNhMfTj_3JlWr60OPYYriY-Tvejoz91g6Qg==
INFO:aishield.connection:Fetching job details for job id gAAAAABj4NUdnijBn1ZMak9NbGuy-kpb-FxHHQji0CbYnr1CJvwY1ieLOrPbkWzQNhMfTj_3JlWr60OPYYriY-Tvejoz91g6Qg==
2023-02-06 10:23:32,621 - INFO - ModelExploration_Status:completed
INFO:aishield.connection:ModelExploration_Status:completed
2023-02-06 10:23:32,626 - INFO - SanityCheck_Status:passed
INFO:aishield.connection:SanityCheck_Status:passed


2023-02-06 10:23:38,480 - INFO - QueryGenerator_Status:completed
INFO:aishield.connection:QueryGenerator_Status:completed


2023-02-06 10:29:01,604 - INFO - VunerabilityEngine_Status:completed
INFO:aishield.connection:VunerabilityEngine_Status:completed


2023-02-06 10:33:38,427 - INFO - DefenseReport_Status:completed
INFO:aishield.connection:DefenseReport_Status:completed
2023-02-06 10:33:38,432 - INFO - Analysis completed for job id gAAAAABj4NUdnijBn1ZMak9NbGuy-kpb-FxHHQji0CbYnr1CJvwY1ieLOrPbkWzQNhMfTj_3JlWr60OPYYriY-Tvejoz91g6Qg==
INFO:aishield.connection:Analysis completed for job id gAAAAABj4NUdnijBn1ZMak9NbGuy-kpb-FxHHQji0CbYnr1CJvwY1ieLOrPbkWzQNhMfTj_3JlWr60OPYYriY-Tvejoz91g6Qg==


job run completed
job status  success


### `Saving the Artifacts and the Reports`

In [ ]:
"""
Description: Creating a directory to save the defense artifacts
"""
OUTPUT_PATH = os.path.join(os.getcwd(),"Output_Artifacts")
make_directory([OUTPUT_PATH])

directory /content/Output_Artifacts created successfully


In [ ]:
"""
Description: Download the Vulnerability Report
"""
if my_status == "success":
    output_conf = ais.OutputConf(report_type=ais.get_type("report", "vulnerability"),
                                 file_format=ais.get_type("file_format", "pdf"),
                                 save_folder_path=OUTPUT_PATH)
        
    my_report = client.save_job_report(job_id=my_job_id, output_config=output_conf)

2023-02-06 10:37:31,352 - INFO - directory /content/Output_Artifacts already exist
INFO:aishield.utils.util:directory /content/Output_Artifacts already exist
2023-02-06 10:37:33,979 - INFO - vulnerability_20230206_1037.pdf is saved in /content/Output_Artifacts
INFO:aishield.connection:vulnerability_20230206_1037.pdf is saved in /content/Output_Artifacts


In [ ]:
"""
Description: Download the Defense Reports
"""
if my_status == "success":
    output_conf = ais.OutputConf(report_type=ais.get_type("report", "defense"),
                                 file_format=ais.get_type("file_format", "pdf"),
                                 save_folder_path=OUTPUT_PATH)
        
    my_report = client.save_job_report(job_id=my_job_id, output_config=output_conf)

2023-02-06 10:37:34,022 - INFO - directory /content/Output_Artifacts already exist
INFO:aishield.utils.util:directory /content/Output_Artifacts already exist
2023-02-06 10:37:35,674 - INFO - defense_20230206_1037.pdf is saved in /content/Output_Artifacts
INFO:aishield.connection:defense_20230206_1037.pdf is saved in /content/Output_Artifacts


In [ ]:
"""
Description: Download the Defense artifacts: Model
"""
if my_status == "success":
    output_conf = ais.OutputConf(report_type=ais.get_type("report", "defense_artifact"),
                                 file_format=ais.get_type("file_format", "pdf"),
                                 save_folder_path=OUTPUT_PATH)
        
    my_report = client.save_job_report(job_id=my_job_id, output_config=output_conf)

2023-02-06 10:37:49,210 - INFO - directory /content/Output_Artifacts already exist
INFO:aishield.utils.util:directory /content/Output_Artifacts already exist
2023-02-06 10:37:50,522 - INFO - defense_artifact_20230206_1037.zip is saved in /content/Output_Artifacts
INFO:aishield.connection:defense_artifact_20230206_1037.zip is saved in /content/Output_Artifacts


In [ ]:
"""
Description: Download the Attack Samples
"""
if my_status == "success":
    output_conf = ais.OutputConf(report_type=ais.get_type("report", "attack_samples"),
                                 save_folder_path=OUTPUT_PATH)
        
    my_report = client.save_job_report(job_id=my_job_id, output_config=output_conf)

2023-02-06 10:37:50,545 - INFO - directory /content/Output_Artifacts already exist
INFO:aishield.utils.util:directory /content/Output_Artifacts already exist
2023-02-06 10:37:51,442 - INFO - attack_samples_20230206_1037.zip is saved in /content/Output_Artifacts
INFO:aishield.connection:attack_samples_20230206_1037.zip is saved in /content/Output_Artifacts
